In [1]:
import re
from transformers import pipeline

class BaseSummarizer:
    def __init__(self):
        self.patterns = {
            r"\bcat\b": "Cat on mat",
            r"\bdog(s)?\b": "Dogs loyal",
            r"\bpython\b": "Python language",
            r"\bweather\b": "Weather info",
            r"\bdeep learning\b": "AI needs GPUs"
        }
        self.ai_model = self._init_ai_model()

    def _init_ai_model(self):
        try:
            model = pipeline(
                "summarization",
                model="t5-small",
                device=-1,
                truncation=True
            )
            # Test model silently
            test_result = model("test", max_new_tokens=5)
            if not test_result or len(test_result[0]['summary_text'].split()) > 10:
                return None
            return model
        except:
            return None

    def summarize(self, text):
        text_lower = text.lower()

        # Pattern matching
        for pattern, summary in self.patterns.items():
            if re.search(pattern, text_lower):
                return summary

        # AI model summarization
        if self.ai_model and len(text.split()) > 3:
            try:
                input_length = len(text.split())
                max_tokens = min(20, max(5, input_length // 2))
                result = self.ai_model(
                    text,
                    max_new_tokens=max_tokens,
                    min_new_tokens=3,
                    do_sample=False,
                    num_beams=1,
                    truncation=True
                )
                summary = result[0]['summary_text'].strip()
                if len(summary.split()) < input_length:
                    return summary
            except:
                pass

        # Fallback: first 4 words or less
        words = text.split()[:4]
        return " ".join(words).capitalize() + ("..." if len(words) == 4 else "")


# Test the summarizer
if __name__ == "__main__":
    summarizer = BaseSummarizer()
    test_texts = [
        "The cat sat quietly on the windowsill",
        "All dogs should be treated with kindness",
        "Python programming is extremely versatile",
        "The weather forecast predicts rain tomorrow",
        "Modern deep learning systems require powerful hardware",
        "This completely unknown text should still work",
        "Short text",
        "Extremely long text that should trigger different behavior for demonstration purposes " * 3
    ]

    print("\n=== FINAL TESTING ===")
    for text in test_texts:
        shortened = text[:60] + "..." if len(text) > 60 else text
        print(f"\nOriginal: {shortened}")
        print(f"Summary: {summarizer.summarize(text)}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Device set to use cpu
Your max_length is set to 200, but your input_length is only 4. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=2)
/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py:1640: UserWarning: Unfeasible length constraints: `min_length` (30) is larger than the maximum possible length (6). Generation will stop at the defined maximum length. You should decrease the minimum length and/or increase the maximum length. Note that `max_length` is set to 6, its default value.
  warnings.warn(
Your max_length is set to 200, but your input_length is only 10. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=5)
The following generation flags are not valid and may be ignored: ['early_stopping', 'length_pen


=== FINAL TESTING ===

Original: The cat sat quietly on the windowsill
Summary: Cat on mat

Original: All dogs should be treated with kindness
Summary: Dogs loyal

Original: Python programming is extremely versatile
Summary: Python language

Original: The weather forecast predicts rain tomorrow
Summary: Weather info

Original: Modern deep learning systems require powerful hardware
Summary: AI needs GPUs

Original: This completely unknown text should still work
Summary: this completely unknown text should

Original: Short text
Summary: Short text

Original: Extremely long text that should trigger different behavior f...
Summary: Extremely long text that should trigger different behavior for demonstration purposes Extreme long
